In [17]:
#imports
import numpy as np 
import pandas as pd 
from scipy import stats
from matplotlib import pyplot as plt
#from pandas_profiling import ProfileReport
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.utils import resample


#load data
train_x = pd.read_csv('TrainingSetValues.csv')
train_y = pd.read_csv('TrainingSetLabels.csv')
train = pd.merge(train_y,train_x, how='left', left_on = 'id', right_on='id')

test_x = pd.read_csv('TestSetValues.csv')

In [18]:
#profile = ProfileReport(train)
#profile.to_notebook_iframe()

In [19]:
#denote columns to keep
cols_keep = ['id', 'status_group', 'amount_tsh', 'funder', 'gps_height', 'installer', 'basin', 'region_code', 'population', 'public_meeting', 'scheme_management', 'permit', 'construction_year', 'extraction_type_class', 'management_group', 'payment', 'quality_group', 'quantity', 'source_type', 'source_class', 'waterpoint_type', 'ward']

#create dataframe keeping only these columns
train_final = train[cols_keep]

In [20]:
#find wards with greater than 70% of pumps broken
df = train[['ward','status_group']]

#find count of pumps by ward
ward_count = df.groupby(['ward'])['ward'].count().reset_index(name="ward_total")
#find count of broken pumps by ward
ward_count_broken = df.query('status_group == "non functional"').groupby(['ward'])['ward'].count().reset_index(name="ward_broken")
#join counts
ward_count = pd.merge(ward_count, ward_count_broken)
#find % broken by ward
ward_count['percent_broken'] = ward_count['ward_broken'] / ward_count['ward_total']

#generate an index of wards w/ more than 70% pumps broken
ward_over_70 = ward_count.groupby('ward')['percent_broken'].max().apply(lambda g: g>0.7)
ward_over_70 = ward_over_70[ward_over_70].index

#relabel all wards 1 if greater than 70% broken, 0 if else
train_final['ward'] = train_final.apply(lambda row: 1 if row['ward'] in ward_over_70 else 0, axis = 1).astype('bool')


/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [21]:
#replace 0 w/ other in funder column
train_final['funder'] = train_final['funder'].replace('0', 'other')

#generate an index of funders w/ more than 500 wells
funder_over_500 = train_final.groupby('funder')['status_group'].count().apply(lambda g: g>500)
funder_over_500 = funder_over_500[funder_over_500].index

#relabel all funders w/ less than 500 wells other
train_final['funder'] = train_final.apply(lambda row: row['funder'] if row['funder'] in funder_over_500 else 'other', axis = 1)

#replace 0 w/ other in installer column
train_final['installer'] = train_final['installer'].replace('0', 'other')

#generate an index of funders w/ more than 500 wells
installer_over_500 = train_final.groupby('installer')['status_group'].count().apply(lambda g: g>500)
installer_over_500 = installer_over_500[installer_over_500].index

#relabel all funders w/ less than 500 wells other
train_final['installer'] = train_final.apply(lambda row: row['installer'] if row['installer'] in funder_over_500 else 'other', axis = 1)

/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarn

In [22]:
#drop duplicates
train_final = train_final.drop_duplicates()
#change public meeting and permit to bool
train_final[['public_meeting', 'permit']] = train_final[['public_meeting', 'permit']].astype('category')

In [23]:
#replace management under 1k w/ other
train_final['scheme_management'] = train_final['scheme_management'].replace(['SWC', 'Trust', 'None'], 'Other')
#fill na management w/ missing
train_final['scheme_management'] = train_final['scheme_management'].fillna('Missing')

In [24]:
train_final.to_csv('train_final.csv', index=False)

In [25]:
#numerically encode categorical variables
le = LabelEncoder()
train_final = train_final.apply(lambda col: le.fit_transform(col) if col.dtype == 'object' else col)

In [26]:
one_hot_list = ['funder', 'installer', 'basin', 'region_code', 'scheme_management', 'extraction_type_class', 'management_group', 'payment', 'quality_group', 'quantity', 'source_type', 'source_class', 'waterpoint_type']

In [27]:
# generate binary values using get_dummies
dum_df = pd.get_dummies(train_final, columns=one_hot_list)

In [28]:
dum_df.to_csv('train_dummy_final.csv', index=False)

In [29]:
cols_keep_test = ['id', 'amount_tsh', 'funder', 'gps_height', 'installer', 'basin', 'region_code', 'population', 'public_meeting', 'scheme_management', 'permit', 'construction_year', 'extraction_type_class', 'management_group', 'payment', 'quality_group', 'quantity', 'source_type', 'source_class', 'waterpoint_type', 'ward']

test_x = test_x[cols_keep_test]

#relabel all wards 1 if greater than 70% broken, 0 if else
test_x['ward'] = test_x.apply(lambda row: 1 if row['ward'] in ward_over_70 else 0, axis = 1).astype('bool')

#relabel all funders w/ less than 500 wells other
test_x['installer'] = test_x.apply(lambda row: row['installer'] if row['installer'] in funder_over_500 else 'other', axis = 1)


#relabel all funders w/ less than 500 wells other
test_x['funder'] = test_x.apply(lambda row: row['funder'] if row['funder'] in funder_over_500 else 'other', axis = 1)


#drop duplicates
test_x = test_x.drop_duplicates()

#change public meeting and permit to bool
test_x[['public_meeting', 'permit']] = test_x[['public_meeting', 'permit']].astype('category')

#replace management under 1k w/ other
test_x['scheme_management'] = test_x['scheme_management'].replace(['SWC', 'Trust', 'None'], 'Other')
#fill na management w/ missing
test_x['scheme_management'] = test_x['scheme_management'].fillna('Missing')

#to_csv
test_x.to_csv('test_final.csv', index=False)

test_x = test_x.apply(lambda col: le.fit_transform(col) if col.dtype == 'object' else col)

# generate binary values using get_dummies
test_dum_df = pd.get_dummies(test_x, columns=one_hot_list)

test_dum_df.to_csv('test_dummy_final.csv', index=False)

In [30]:
dum_df.status_group.value_counts()

0    32259
2    22824
1     4317
Name: status_group, dtype: int64

In [40]:
#seperate each class for resampling
dum_df_0 = dum_df[dum_df.status_group==0]
dum_df_1 = dum_df[dum_df.status_group==1]
dum_df_2 = dum_df[dum_df.status_group==2]

#resample class 1 and 2 up to size of class 0
dum_df_1 = resample(dum_df_1, replace=True, n_samples=32259, random_state=20)
dum_df_2 = resample(dum_df_2, replace=True, n_samples=32259, random_state=20)

#concat dataframes
dum_df_list = [dum_df_0, dum_df_1, dum_df_2]
dum_df_upsampled = pd.concat(dum_df_list)
dum_df_upsampled.to_csv('train_dummy_upsampled_final.csv', index=False)